In [49]:
import numpy as np
import scipy.io
import scipy.signal
import os
import mne
import matplotlib.pyplot as plt
import pandas
import math
from pymatreader import read_mat

In [50]:
dataset_folder = 'dataset'
dataset_files = []

for file in os.listdir(dataset_folder):
    dataset_files.append(os.path.join(dataset_folder, file))

print('Number of files: ', len(dataset_files))

Number of files:  18


In [51]:
bci_exp = read_mat(dataset_files[0])['bciexp'] # read only bci_exp object
eeg_data = bci_exp['data']
print(eeg_data.shape)

downsampling_fact = 5 # 250 / 5 = 50 Hz
passband_cutoff_freq = np.array([1.0, 12.5]) # cutoff frequencies
interval_len = .75 # length of analysis interval in seconds
sampling_rate = bci_exp['srate'] # sampling rate in Hz (250)

num_samples = int(interval_len * sampling_rate) - 1

# nStim = sum( diff( sum( bciexp.stim( :, :, 1))) > 0);
num_stimuli = np.sum(np.diff(bci_exp['stim'][:,:,0] > 0))
num_trials = eeg_data.shape[2]

channels_of_interest = ['O9', 'CP1', 'CP2', 'O10', 'P7', 'P3', 'Pz',
            'P4', 'P8', 'PO7', 'PO3', 'Oz', 'PO4', 'PO8']
num_channels = len(channels_of_interest)
channel_indexes = [bci_exp['label'].index(x) for x in channels_of_interest]

lmast_channel = np.char.equal('LMAST', bci_exp['label'])

data = np.zeros(shape=(num_channels, num_samples, num_stimuli, num_trials))
butter_a, butter_b = scipy.signal.butter(2, passband_cutoff_freq / (sampling_rate / 2))
for i in range(num_trials):
    right_data = eeg_data[channel_indexes, :, i] - (eeg_data[lmast_channel, :, i] / 2)
    right_data = scipy.signal.resample(
        scipy.signal.filtfilt()
    )

(30, 4000, 120)


ValueError: Must specify a single critical frequency Wn for lowpass or highpass filter